# Test Code for Kinematic and Dynamic Solvers

This notebook contains test cases for:
1. Forward Kinematics
2. Inverse Kinematics
3. Inverse Dynamics

In [8]:
import numpy as np
import yaml
from kinematic_solver import KinematicSolver
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 1. Load Robot Parameters from YAML

In [ ]:
# Load robot parameters from YAML file
with open('stand_like_robot.yaml', 'r') as file:
    robot_params = yaml.safe_load(file)

# Extract DH parameters, masses, COMs, and inertias
dh_params = [link['dh_params'] for link in robot_params['links'] if link['type'] == 'revolute']
masses = [link['mass'] for link in robot_params['links'] if link['type'] == 'revolute']
coms = [np.array(link['com']) for link in robot_params['links'] if link['type'] == 'revolute']
inertias = [np.array(link['inertia']) for link in robot_params['links'] if link['type'] == 'revolute']

print(f"Number of joints: {len(dh_params)}")
print("\nDH Parameters:")
for i, params in enumerate(dh_params):
    print(f"Joint {i}:", params)

## 2. Initialize Solvers

In [ ]:
# Initialize kinematic and dynamic solvers
kinematic_solver = KinematicSolver(dh_params)
dynamics_solver = InverseDynamicsSolver(dh_params, masses, coms, inertias)

## 3. Test Forward Kinematics

In [ ]:
def test_forward_kinematics():
    # Test case 1: Zero position
    joint_angles = np.zeros(len(dh_params))
    position, orientation = kinematic_solver.forward_kinematics(joint_angles)
    print("Test 1 - Zero Position:")
    print(f"Position: {position}")
    print(f"Orientation: {np.rad2deg(orientation)} degrees\n")
    
    # Test case 2: Random position
    joint_angles = np.random.uniform(-np.pi, np.pi, len(dh_params))
    position, orientation = kinematic_solver.forward_kinematics(joint_angles)
    print("Test 2 - Random Position:")
    print(f"Joint angles: {np.rad2deg(joint_angles)} degrees")
    print(f"Position: {position}")
    print(f"Orientation: {np.rad2deg(orientation)} degrees")
    
    return joint_angles, position, orientation

joint_angles, position, orientation = test_forward_kinematics()

## 4. Test Inverse Kinematics

In [ ]:
def test_inverse_kinematics():
    # Test case 1: Using position from forward kinematics
    target_position = position  # Use position from previous test
    target_orientation = orientation  # Use orientation from previous test
    
    # Solve inverse kinematics
    solved_angles = kinematic_solver.inverse_kinematics(
        target_position,
        target_orientation,
        seed=np.zeros(len(dh_params))
    )
    
    # Verify solution
    solved_position, solved_orientation = kinematic_solver.forward_kinematics(solved_angles)
    position_error = np.linalg.norm(target_position - solved_position)
    orientation_error = np.linalg.norm(target_orientation - solved_orientation)
    
    print("Test 1 - Inverse Kinematics:")
    print(f"Original joint angles: {np.rad2deg(joint_angles)} degrees")
    print(f"Solved joint angles: {np.rad2deg(solved_angles)} degrees")
    print(f"Position error: {position_error}")
    print(f"Orientation error: {orientation_error} radians")
    
    return solved_angles, position_error, orientation_error

solved_angles, pos_error, orient_error = test_inverse_kinematics()

## 5. Test Inverse Dynamics

In [ ]:
def test_inverse_dynamics():
    # Test case 1: Static position (zero velocity and acceleration)
    joint_angles = np.zeros(len(dh_params))
    joint_velocities = np.zeros(len(dh_params))
    joint_accelerations = np.zeros(len(dh_params))
    
    torques = dynamics_solver.inverse_dynamics(
        joint_angles,
        joint_velocities,
        joint_accelerations
    )
    
    print("Test 1 - Static Position:")
    print(f"Required torques: {torques}\n")
    
    # Test case 2: Dynamic motion
    joint_angles = np.random.uniform(-np.pi, np.pi, len(dh_params))
    joint_velocities = np.random.uniform(-1, 1, len(dh_params))
    joint_accelerations = np.random.uniform(-1, 1, len(dh_params))
    
    torques = dynamics_solver.inverse_dynamics(
        joint_angles,
        joint_velocities,
        joint_accelerations
    )
    
    print("Test 2 - Dynamic Motion:")
    print(f"Joint angles: {np.rad2deg(joint_angles)} degrees")
    print(f"Joint velocities: {joint_velocities} rad/s")
    print(f"Joint accelerations: {joint_accelerations} rad/s²")
    print(f"Required torques: {torques}")
    
    return torques

torques = test_inverse_dynamics()

## 6. Visualize Robot Configuration

In [ ]:
def visualize_robot_configuration(joint_angles):
    # Calculate positions of all joints
    positions = []
    T = np.eye(4)
    positions.append(T[:3, 3])  # Base position
    
    for i, (params, theta) in enumerate(zip(dh_params, joint_angles)):
        params['theta'] = theta
        Ti = kinematic_solver._dh_transform(
            params['theta'],
            params['d'],
            params['a'],
            params['alpha']
        )
        T = T @ Ti
        positions.append(T[:3, 3])
    
    # Plot robot configuration
    positions = np.array(positions)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot links
    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 'b-', linewidth=2)
    
    # Plot joints
    ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c='r', s=100)
    
    # Set labels and limits
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Robot Configuration')
    
    # Set equal aspect ratio
    max_range = np.array([
        positions[:, 0].max() - positions[:, 0].min(),
        positions[:, 1].max() - positions[:, 1].min(),
        positions[:, 2].max() - positions[:, 2].min()
    ]).max() / 2.0
    
    mid_x = (positions[:, 0].max() + positions[:, 0].min()) * 0.5
    mid_y = (positions[:, 1].max() + positions[:, 1].min()) * 0.5
    mid_z = (positions[:, 2].max() + positions[:, 2].min()) * 0.5
    
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.show()

# Visualize the robot configuration from the inverse kinematics test
visualize_robot_configuration(solved_angles)